<a href="https://colab.research.google.com/github/saravanamuthu1/sentiment_analysis/blob/main/sentiment_analysis_sractch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
mylist=[]
label=[]
value1=[]
testfile = open('train-v2.tsv', 'r')
for line in testfile:
  value=line.rstrip("\n").split("\t")
  mylist.append(line.rstrip("\n").split("\t"))
  label.append(value[0])
  value1.append(value[1])

In [108]:
print(len(label))
print(len(value1))
print(len(mylist))

80000
80000
80000


In [109]:
train_set,test_set=value1[10000:80000],value1[0:10000]
train_label,test_lable=label[10000:80000],label[0:10000]

In [110]:
def remove_word(list_to_be_removed):
  for i,j in enumerate(list_to_be_removed):
    text = j.replace("@USER", "")
    list_to_be_removed.insert(i,text)
  return list_to_be_removed




tokenize data

In [119]:
dict1={}
dict2={}
for i in mylist:
  if i[0] == "0":
    tweet=i[1].split(" ")
    for word in tweet:
      if word in dict1:
        dict1[word]+=1
      else:
        dict1[word] = 1
  elif i[0] == "1":
    tweet=i[1].split(" ")
    for word in tweet:
      if word in dict2:
        dict2[word]+=1
      else:
        dict2[word] = 1


In [121]:
dict1.pop("@USER")
dict2.pop("@USER")

42688

get the probability